In [ ]:
product = None
upstream = ['01-uniform-files']
input = None

In [ ]:
import prody as pdy
import pandas as pd
import numpy as np
import json

In [ ]:
reference = pd.read_csv(upstream['01-uniform-files']['data'], index_col=None)
reference.chain_reference = reference.chain_reference.apply(lambda x: json.loads(x.replace('\'', '"')))

In [ ]:
def extract_subunits(id, chain_reference, reference):
    pdb = pdy.parsePDB(id + '.pdb')
    
    for chain in pdb.getHierView().iterChains():
        chain_id = chain.getChid()
        subunit_id = chain_reference[chain_id]
        chain_pLDDT = np.mean(chain.select('name CA').getBetas())
        sequence = chain.select('name CA').getSequence()
        reference.append(
            dict(id=id, chain=chain_id, pLDDT=chain_pLDDT, subunit=subunit_id, sequence=sequence)
        )



In [ ]:
chain_index = []
for line in reference.to_dict(orient='records'):
    extract_subunits(line['id'], line['chain_reference'], chain_index)

In [ ]:
pd.DataFrame.from_records(chain_index).to_csv(product['data'], index=None)
